In [ ]:
# install all modules to help with data cleaning and storage
!pip install flatdict==4.0.1
!pip install google-cloud-bigquery
!pip install --upgrade google-cloud-bigquery-storage

In [ ]:
# import get and post requests from lib
from requests import get,post
import os
import flatdict
from google.cloud import bigquery
from google.oauth2 import service_account

In [ ]:
# documentation - https://aviationstack.com/documentation

In [ ]:
# api keys
# os.environ["API_KEY"]
key="a686bdcebdac602cfbb96ff529bd9cb8"
weather_api_key="0dc74d9415a7e5a964398b39942cf3d9"

In [ ]:
# get info on flights
flights_url = "http://api.aviationstack.com/v1/flights"
airlines_url = "http://api.aviationstack.com/v1/airlines"
# urls for api detailing current weather and historical weather
current_weather_url = "http://api.aviationstack.com/v1/current"
historical_weather_url = "http://api.aviationstack.com/v1/historical"

In [ ]:
# helper function to get data from the apis
def get_from_api(url,query,api_key=key):
    return get(url+"?access_key="+api_key+query).json()

In [ ]:
ba_flights = get_from_api(flights_url, "&airline_iata=BA")
# try getting data current weather in new york
london_weather = get_from_api(current_weather_url, "&query=london", weather_api_key)
london_weather